In [0]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer,TfidfVectorizer
from nltk import word_tokenize
import string 
from sklearn.decomposition import nmf
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from nltk.corpus import wordnet
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts,GridSearchCV
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

In [0]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


In [0]:
def text(str):
  str_new = " ".join(re.findall("[a-zA-Z]+", str))
  return (str_new)

In [0]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [0]:
def tokenize(str_new):
  tokens = []
  tokens = nltk.word_tokenize(str_new)
  return (tokens)

In [0]:
def remove(tokens):
  removed = []
  stopword = stopwords.words('english') 
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for w in tokens:
    if (w.lower() not in stopword and w.lower() not in punctuations):
      removed.append(w.lower())
  return (removed)

In [0]:
def tag(removed):
  pos = []
  pos.append(nltk.pos_tag(removed))
  return (pos)

In [0]:
def lemmatize(pos):
  lem = WordNetLemmatizer()
  lemma = []
  for word in pos:
    for w in word:
      pos_value = ""
      if (w[1].startswith('J')):
        pos_value = wordnet.ADJ
      elif (w[1].startswith('V')):
        pos_value = wordnet.VERB
      elif (w[1].startswith('N')):
        pos_value = wordnet.NOUN
      elif (w[1].startswith('R')):
        pos_value = wordnet.ADV 
      else:
        continue
      lemma.append(lem.lemmatize(w[0],pos_value))
  return (lemma)

In [0]:
def string(lemma):
  str1 = ' '.join([elem for elem in lemma]) 
  return (str1)

In [0]:
def to_sent(sents):
  doc = []
  for sent in sents:
    doc.append(sent)
    return (doc)

In [0]:
def TFiDF(doc):
  vectorizer = TfidfVectorizer()
  vectors = vectorizer.fit_transform(doc)
  feature_names = vectorizer.get_feature_names()
  dense = vectors.todense()
  denselist = dense.tolist()
  df = pd.DataFrame(denselist, columns=feature_names)
  return (df)

In [0]:
steps = [("text", text()), ("tokenize", tokenize()), ("remove", remove()), ("tag", tag()), ("lemmatize", lemmatize()), ("string", string()), ("to_sent", to_sent()), ("TFiDF", TFiDF())]

In [0]:
preprocess_pipeline = Pipeline(steps)

In [0]:
data = pd.read_csv("/content/Womens Clothing E-Commerce Reviews.csv")

In [0]:
data.shape

(20333, 11)

In [0]:
data.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33.0,NaN,Absolutely wonderful - silky and sexy and comf...,4.0,1.0,0.0,Initmates,Intimate,Intimates
1,1,1080,34.0,NaN,Love this dress! it's sooo pretty. i happene...,5.0,1.0,4.0,General,Dresses,Dresses
2,2,1077,60.0,Some major design flaws,I had such high hopes for this dress and reall...,3.0,0.0,0.0,General,Dresses,Dresses
3,3,1049,50.0,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5.0,1.0,0.0,General Petite,Bottoms,Pants
4,4,847,47.0,Flattering shirt,This shirt is very flattering to all due to th...,5.0,1.0,6.0,General,Tops,Blouses


In [0]:
data.drop("Unnamed: 0",1,inplace=True)

In [0]:
data.fillna(" ",inplace = True)

In [0]:
data['Text'] = data['Title'].str.cat(data['Review Text'], sep =" ") 

In [0]:
data.drop(["Title", "Review Text"],1, inplace=True)

In [0]:
data.drop(["Recommended IND"],1, inplace=True)

In [0]:
division = pd.get_dummies(data["Division Name"])
department = pd.get_dummies(data["Department Name"])
class_ = pd.get_dummies(data["Class Name"])

In [0]:
division.drop([" "],1, inplace=True)
department.drop([" "],1, inplace=True)
class_.drop([" "],1, inplace=True)

In [0]:
data2 = pd.concat([data, division, department, class_],1)

In [0]:
data2.drop(["Division Name", "Department Name", "Class Name"],1, inplace = True)

In [0]:
data2.head()

,Clothing ID,Age,Rating,Positive Feedback Count,Text,General,General Petite,Initmates,Bottoms,Dresses,Intimate,Jackets,Tops,Trend,Blouses,Casual bottoms,Chemises,Dresses,Fine gauge,Intimates,Jackets,Jeans,Knits,Layering,Legwear,Lounge,Outerwear,Pants,Shorts,Skirts,Sleep,Sweaters,Swim,Trend
0,767,33,4,0,Absolutely wonderful - silky and sexy and co...,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1080,34,5,4,Love this dress! it's sooo pretty. i happe...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1077,60,3,0,Some major design flaws I had such high hopes ...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1049,50,5,0,"My favorite buy! I love, love, love this jumps...",0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,847,47,5,6,Flattering shirt This shirt is very flattering...,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
data2.columns

Index(['Clothing ID', 'Age', 'Rating', 'Positive Feedback Count', 'Text',
       'General', 'General Petite', 'Initmates', 'Bottoms', 'Dresses',
       'Intimate', 'Jackets', 'Tops', 'Trend', 'Blouses', 'Casual bottoms',
       'Chemises', 'Dresses', 'Fine gauge', 'Intimates', 'Jackets', 'Jeans',
       'Knits', 'Layering', 'Legwear', 'Lounge', 'Outerwear', 'Pants',
       'Shorts', 'Skirts', 'Sleep', 'Sweaters', 'Swim', 'Trend'],
      dtype='object')

In [0]:
preprocess_pipeline(data2["Text"])

In [0]:
X_train, X_test, Y_train, Y_test = tts(X,Y,test_size = 0.25,random_state=42)

In [0]:
nmf_a = nmf(4, random_state = 42, alpha = 0.1, l1_ratio = 0.5, init = "nndsvd")

In [0]:
nmf_a.fit()

In [0]:
model = GaussianNB()

In [0]:
model.fit(X_train,Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
Y_pred = model.predict(X_test)

In [0]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           1       0.05      0.39      0.08       194
           2       0.08      0.20      0.11       407
           3       0.13      0.17      0.15       766
           4       0.21      0.18      0.20      1259
           5       0.61      0.20      0.30      3246

    accuracy                           0.20      5872
   macro avg       0.22      0.23      0.17      5872
weighted avg       0.41      0.20      0.24      5872



In [0]:
log = LogisticRegression()

In [0]:
log.fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
Y_pred1 = log.predict(X_test)

In [0]:
print(classification_report(Y_test,Y_pred1))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       194
           2       0.12      0.00      0.00       407
           3       0.53      0.01      0.02       766
           4       0.29      0.11      0.16      1259
           5       0.58      0.96      0.72      3246

    accuracy                           0.56      5872
   macro avg       0.31      0.22      0.18      5872
weighted avg       0.46      0.56      0.44      5872



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
